In [1]:
from decimal import Decimal, getcontext
from vector import Vector

getcontext().prec = 30


class Line:

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0'] * self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()
                
    def set_basepoint(self):
        try:
            n = self.normal_vector
            # print(n)  # Vector: (Decimal('4.046'), Decimal('2.836'))
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n.coordinates)
            # print(initial_index)  # 0

            initial_coefficient = n.coordinates[initial_index]
            # print(initial_coefficient)  # 4.046
            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e

    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i == initial_index)) + 'x_{}'.format(i + 1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output

    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)

    def is_parallel_to(self, ell):
        
        n1 = self.normal_vector
        n2 = ell.normal_vector
        
        return n1.is_parallel_to(n2)

    def __eq__(self, ell):
        
        if self.normal_vector.is_zero(): 
            if not ell.normal_vector.is_zero():
                return False
            else:
                diff = self.constant_term - ell.constant_term
                return MyDecimal(diff).is_near_zero()
            
        elif ell.normal_vector.is_zero():
            return False

        if not self.is_parallel_to(ell):
            return False
        
        x0 = self.basepoint
        y0 = ell.basepoint

        basepoint_difference = x0.minus(y0)
        
        n = self.normal_vector
        
        return basepoint_difference.is_orthogonal_to(n)

    def intersection_with(self, ell):
        try:
            A, B = self.normal_vector.coordinates
            C, D = ell.normal_vector.coordinates
            k1 = self.constant_term
            k2 = ell.constant_term

            x_numerator = D*k1 - B*k2
            y_numerator = -C*k1 + A*k2
            one_over_denom = Decimal('1')/(A*D - B*C)

            return Vector([x_numerator, y_numerator]).times_scalar(one_over_denom)
        
        except ZeroDivisionError:
            if self == ell:
                return self
            else:
                return None
            

class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps

In [2]:
# first system
# 4.046x + 2.836y = 1.21
# 10.115x + 7.09y = 3.025

ell1 = Line(Vector([4.046, 2.836]), 1.21)
ell2 = Line(Vector([10.115, 7.09]), 3.025)

print('Intersection 1: {}'.format(ell1.intersection_with(ell2)))

Intersection 1: Vector: (Decimal('0.1245112162996006339844193441'), Decimal('0.2490224326001281252702121254'))


In [3]:
# second system
# 7.204x + 3.182y = 8.68
# 8.172x + 4.114y = 9.883

ell1 = Line(Vector([7.204, 3.182]), 8.68)
ell2 = Line(Vector([8.172, 4.114]), 9.883)

print('Intersection 2: {}'.format(ell1.intersection_with(ell2)))

Intersection 2: Vector: (Decimal('1.172776635464641558337360230'), Decimal('0.07269551166333194287712771066'))


In [4]:
# third system
# 1.182x + 5.562y = 6.744
# 1.773x + 8.343y = 9.525

ell1 = Line(Vector([1.182, 5.562]), 6.744)
ell2 = Line(Vector([1.773, 8.343]), 9.525)

print('Intersection 3: {}'.format(ell1.intersection_with(ell2)))

Intersection 3: Vector: (Decimal('-6262255281855965.866642393151'), Decimal('1330813689887406.985144068456'))
